## 1. Problem Formulation

### 1.1 Binary Classification Task

**Objective**: Learn a function that maps nutritional feature vectors to binary fitness labels.

$$f: \mathbb{R}^d \to \{0, 1\}$$

**Input Space**: $\mathcal{X} = \mathbb{R}^d$, where $d = 9$ (after feature selection)

**Output Space**: $\mathcal{Y} = \{0, 1\}$, where:
- $y = 1$: Food is "fit" (nutritionally favorable)
- $y = 0$: Food is "unfit" (nutritionally unfavorable)

**Training Data**:

$$\mathcal{D} = \{(\mathbf{x}_i, y_i)\}_{i=1}^{n}$$

where $n = 164$ training samples

**Test Data**:

$$\mathcal{D}_{\text{test}} = \{(\mathbf{x}_j, y_j)\}_{j=1}^{m}$$

where $m = 41$ test samples

**Goal**: Minimize expected 0-1 loss over the data distribution:

$$f^* = \arg\min_{f \in \mathcal{F}} \mathbb{E}_{(\mathbf{x}, y) \sim P}[\mathbb{1}(f(\mathbf{x}) \neq y)]$$

This objective seeks the optimal classifier that minimizes prediction errors on unseen data.

## 2. Data Preprocessing Pipeline

### 2.1 Missing Value Imputation

**Input**: Raw feature matrix $\mathbf{X}_{\text{raw}} \in \mathbb{R}^{n \times d_{\text{raw}}}$ with missing entries (NaN)

**Transformation**: Median imputation operator $\phi_{\text{impute}}: \mathbb{R}^{n \times d_{\text{raw}}} \to \mathbb{R}^{n \times d_{\text{raw}}}$

**Definition**: For each feature column $\mathbf{x}_{\cdot, j}$:

$$\phi_{\text{impute}}(\mathbf{x}_{\cdot, j}) = \begin{cases}
x_{i,j} & \text{if } x_{i,j} \neq \text{NaN} \\
\tilde{x}_j & \text{if } x_{i,j} = \text{NaN}
\end{cases}$$

where $\tilde{x}_j = \text{median}(\{x_{i,j} : x_{i,j} \neq \text{NaN}\})$

**Explanation**: Missing values are replaced with the median of observed values for that feature. This provides robustness against outliers and maintains the distribution of the data.

### 2.2 Feature Engineering

**Input**: Imputed features $\mathbf{X}_{\text{imputed}} \in \mathbb{R}^{n \times 8}$

**Transformation**: Feature engineering operator $\phi_{\text{engineer}}: \mathbb{R}^{8} \to \mathbb{R}^{13}$

Five engineered features are created to capture domain-specific nutritional relationships:

**1. Nutrient Density**:

$$x_{\text{nutrient\_density}} = \frac{x_{\text{protein}}}{x_{\text{calories}} + 1}$$

Measures protein content relative to caloric density. Higher values indicate more protein per calorie, which is favorable for fitness.

**2. Protein Ratio**:

$$x_{\text{protein\_ratio}} = \frac{x_{\text{protein}}}{x_{\text{carbs}} + x_{\text{fat}} + 1}$$

Quantifies the proportion of protein relative to other macronutrients. High protein-to-macronutrient ratios are associated with muscle maintenance and satiety.

**3. Carb-Fat Ratio**:

$$x_{\text{carb\_fat\_ratio}} = \frac{x_{\text{carbs}}}{x_{\text{fat}} + 1}$$

Captures the balance between carbohydrates and fats. Note: This feature was ultimately excluded during feature selection.

**4. Energy Density**:

$$x_{\text{energy\_density}} = \frac{x_{\text{calories}}}{100}$$

Normalizes calories to a standard portion size (per 100g), enabling comparison across different food quantities.

**5. Micronutrient Score**:

$$x_{\text{micronutrient\_score}} = x_{\text{iron}} + x_{\text{vitamin\_c}}$$

Aggregates key micronutrient content. Iron and Vitamin C are essential nutrients that signal overall nutritional quality.

**Note**: The $+1$ terms in denominators prevent division by zero and provide numerical stability.

**Augmented Feature Vector**:

$$\mathbf{x}_{\text{engineered}} = [\mathbf{x}_{\text{raw}}; \mathbf{x}_{\text{derived}}] \in \mathbb{R}^{13}$$

where $[\cdot; \cdot]$ denotes vector concatenation. The original 8 features are combined with 5 derived features to create a 13-dimensional feature space.

### 2.3 Categorical Encoding

**Input**: Engineered features with categorical variable $c \in \mathcal{C}$ (food category)

where $|\mathcal{C}| = 53$ unique categories

**Transformation**: One-hot encoding operator $\phi_{\text{encode}}: \mathcal{C} \to \{0, 1\}^{53}$

**Definition**:

$$\phi_{\text{encode}}(c) = [e_1, e_2, \ldots, e_{53}], \quad e_k = \begin{cases}
1 & \text{if } c = \text{category}_k \\
0 & \text{otherwise}
\end{cases}$$

Each food category is converted into a binary indicator vector. Only one element is 1 (the category of the food), while all others are 0.

**Augmented Feature Vector** (including dietary flags):

$$\mathbf{x}_{\text{encoded}} = [\mathbf{x}_{\text{engineered}}; \phi_{\text{encode}}(c); \mathbf{x}_{\text{dietary}}] \in \mathbb{R}^{69}$$

where $\mathbf{x}_{\text{dietary}} = [x_{\text{glutenfree}}, x_{\text{nutfree}}, x_{\text{vegan}}] \in \{0, 1\}^3$

The final encoded feature space has 69 dimensions: 13 nutritional features + 53 category indicators + 3 dietary flags.

### 2.4 Target Label Generation

**Input**: Raw nutritional features

$$\mathbf{x} = [x_{\text{protein}}, x_{\text{vitamin\_c}}, x_{\text{iron}}, x_{\text{calories}}, x_{\text{fat}}]$$

**Transformation**: Adaptive labeling function $\phi_{\text{label}}: \mathbb{R}^5 \to \{0, 1\}$

**Fitness Score**:

$$S_{\text{fitness}}(\mathbf{x}) = 2 \cdot x_{\text{protein}} + 1.5 \cdot x_{\text{vitamin\_c}} + 1.5 \cdot x_{\text{iron}} - 0.5 \cdot x_{\text{calories}} - 0.8 \cdot x_{\text{fat}}$$

This weighted score favors:
- High protein content (coefficient 2.0)
- High micronutrients: Vitamin C and Iron (coefficients 1.5 each)
- Low calories and fat (negative coefficients -0.5 and -0.8)

**Percentile Threshold** (70th percentile):

$$\tau = Q_{0.70}(\{S_{\text{fitness}}(\mathbf{x}_i)\}_{i=1}^{n})$$

The threshold is determined adaptively from the data distribution, ensuring balanced class proportions.

**Binary Label**:

$$y = \phi_{\text{label}}(\mathbf{x}) = \begin{cases}
1 & \text{if } S_{\text{fitness}}(\mathbf{x}) \geq \tau \\
0 & \text{otherwise}
\end{cases}$$

Foods scoring above the 70th percentile are labeled "fit" (1), others as "unfit" (0).

## 3. Feature Selection

### 3.1 Chi-Squared Test

**Input**: Encoded feature matrix $\mathbf{X} \in \mathbb{R}^{n \times 69}$, target labels $\mathbf{y} \in \{0, 1\}^n$

**Objective**: Select $k = 9$ features with highest chi-squared scores.

**Chi-Squared Statistic** (for feature $j$):

$$\chi^2_j = \sum_{i=1}^{n} \sum_{c=0}^{1} \frac{(O_{ic}^{(j)} - E_{ic}^{(j)})^2}{E_{ic}^{(j)}}$$

where:
- $O_{ic}^{(j)}$ = Observed frequency of feature bin $i$ in class $c$
- $E_{ic}^{(j)}$ = Expected frequency under independence: $E_{ic}^{(j)} = \frac{n_i \cdot n_c}{n}$

The chi-squared test measures the dependence between each feature and the target variable. Higher scores indicate stronger statistical association.

**Feature Selection Operator** $\phi_{\text{select}}: \mathbb{R}^{69} \to \mathbb{R}^{9}$:

$$\phi_{\text{select}}(\mathbf{x}) = [\mathbf{x}_{j_1}, \mathbf{x}_{j_2}, \ldots, \mathbf{x}_{j_9}]$$

where $j_1, j_2, \ldots, j_9$ are indices of features with top 9 chi-squared scores:

$$\{j_1, \ldots, j_9\} = \arg\max_{|S|=9, S \subseteq \{1,\ldots,69\}} \sum_{j \in S} \chi^2_j$$

This greedy selection retains only the most informative features for classification.

**Selected Features** (ordered by $\chi^2$ score):

$$\mathbf{x}_{\text{selected}} = [x_{\text{vitamin\_c}}, x_{\text{nutrient\_density}}, x_{\text{micronutrient\_score}}, x_{\text{calories}}, x_{\text{category\_beverages}}, x_{\text{fat}}, x_{\text{category\_chips}}, x_{\text{protein\_ratio}}, x_{\text{energy\_density}}]$$

The final 9 features include:
- 3 raw nutritional features (vitamin_c, calories, fat)
- 4 engineered features (nutrient_density, micronutrient_score, protein_ratio, energy_density)
- 2 category indicators (beverages, chips)

## 4. Random Forest Classifier

### 4.1 Decision Tree Construction

**Input**: Training data $\mathcal{D}_t = \{(\mathbf{x}_i, y_i)\}_{i=1}^{n_t}$ (bootstrap sample)

**Tree Structure**: Binary tree $h_t: \mathbb{R}^9 \to \{0, 1\}$

Each tree recursively partitions the feature space via binary splits to create homogeneous leaf nodes.

**Gini Impurity** (for node $N$):

$$G(N) = 1 - \sum_{c=0}^{1} p_c^2$$

where $p_c = \frac{1}{|\mathcal{S}_N|} \sum_{(\mathbf{x}, y) \in \mathcal{S}_N} \mathbb{1}(y = c)$

For binary classification:

$$G(N) = 2p_0 p_1 = 2 p_1 (1 - p_1)$$

Gini impurity measures the heterogeneity of labels in a node. Pure nodes (all samples same class) have $G(N) = 0$.

**Optimal Split** (at node $N$):

$$(f^*, \tau^*) = \arg\max_{f \in F_N, \tau \in \mathbb{R}} \Delta G(N, f, \tau)$$

where:
- $F_N$: Random subset of 3 features (out of 9) - feature bagging for decorrelation
- $\Delta G(N, f, \tau)$: Gini gain from split

**Gini Gain**:

$$\Delta G(N, f, \tau) = G(N) - \left( \frac{|\mathcal{S}_{N_L}|}{|\mathcal{S}_N|} G(N_L) + \frac{|\mathcal{S}_{N_R}|}{|\mathcal{S}_N|} G(N_R) \right)$$

where:
- $N_L$: Left child (samples with $x_f \leq \tau$)
- $N_R$: Right child (samples with $x_f > \tau$)

The split maximizes the reduction in impurity weighted by the proportion of samples sent to each child.

**Stopping Criteria**: Tree construction terminates at node $N$ if any condition is met:

1. **Max Depth**: $\text{depth}(N) = 5$ (prevents overfitting)
2. **Min Samples Split**: $|\mathcal{S}_N| < 5$ (insufficient samples to split)
3. **Min Samples Leaf**: Would create leaf with $<3$ samples (ensures statistical reliability)
4. **Pure Node**: $G(N) = 0$ (all samples same class)

These hyperparameters were optimized via GridSearchCV for the small dataset (n=164).

**Leaf Prediction**:

$$\hat{y}_N = \arg\max_{c \in \{0, 1\}} \sum_{(\mathbf{x}, y) \in \mathcal{S}_N} \mathbb{1}(y = c)$$

The predicted class is the majority class among samples reaching the leaf.

**Leaf Probability Estimate**:

$$P_N(y = 1) = \frac{1}{|\mathcal{S}_N|} \sum_{(\mathbf{x}, y) \in \mathcal{S}_N} \mathbb{1}(y = 1)$$

The proportion of positive class samples provides a probability estimate for classification confidence.

### 4.2 Bootstrap Sampling

**Training Set**: $\mathcal{D}_{\text{train}} = \{(\mathbf{x}_i, y_i)\}_{i=1}^{n}$, $n = 164$

**Bootstrap Sample for Tree $t$**:

$$\mathcal{D}_t = \{(\mathbf{x}_{i_1}, y_{i_1}), (\mathbf{x}_{i_2}, y_{i_2}), \ldots, (\mathbf{x}_{i_n}, y_{i_n})\}$$

where $i_j \sim \text{Discrete Uniform}(\{1, \ldots, n\})$ (sampling with replacement)

Each tree sees a different random sample of the training data, promoting diversity in the ensemble.

**Expected Unique Samples**:

$$\mathbb{E}[|\{\text{unique samples in } \mathcal{D}_t\}|] \approx n \left(1 - \frac{1}{e}\right) \approx 0.632n \approx 104 \text{ samples}$$

On average, each bootstrap sample contains about 63.2% unique samples, with some samples appearing multiple times and others not at all. This creates natural train-validation splits within each tree.

### 4.3 Ensemble Prediction

**Forest Definition**: Collection of $T = 50$ trees $\{h_1, h_2, \ldots, h_T\}$

**Class Probability Estimation**:

$$P_{\text{RF}}(y = 1 \mid \mathbf{x}) = \frac{1}{T} \sum_{t=1}^{T} P_t(y = 1 \mid \mathbf{x})$$

where $P_t(y = 1 \mid \mathbf{x})$ is the probability estimate from tree $t$ (leaf node probability).

The ensemble averages probability estimates across all trees to produce a smooth, calibrated prediction.

**Classification Rule**:

$$\hat{y}_{\text{RF}}(\mathbf{x}) = \begin{cases}
1 & \text{if } P_{\text{RF}}(y = 1 \mid \mathbf{x}) \geq 0.5 \\
0 & \text{otherwise}
\end{cases}$$

**Alternative (Majority Vote)**:

$$\hat{y}_{\text{RF}}(\mathbf{x}) = \arg\max_{c \in \{0, 1\}} \sum_{t=1}^{T} \mathbb{1}(h_t(\mathbf{x}) = c)$$

Both formulations are equivalent for binary classification with threshold 0.5.

### 4.4 Feature Importance

**Gini Importance for Feature $f$**:

$$I(f) = \frac{1}{T} \sum_{t=1}^{T} \sum_{N \in \text{Nodes}_t(f)} \frac{|\mathcal{S}_N|}{n} \cdot \Delta G(N, f, \tau_N)$$

where $\text{Nodes}_t(f)$ is the set of nodes in tree $t$ that split on feature $f$.

Feature importance aggregates the total Gini gain contributed by each feature across all splits in all trees, weighted by the number of samples.

**Normalized Importance**:

$$I_{\text{norm}}(f) = \frac{I(f)}{\sum_{f'=1}^{9} I(f')}$$

Normalization ensures importances sum to 1, allowing interpretation as relative contributions.

**NutriSolve Feature Importances**:

$$\mathbf{I} = \begin{bmatrix}
I(\text{vitamin\_c}) \\
I(\text{micronutrient\_score}) \\
I(\text{nutrient\_density}) \\
\vdots \\
I(\text{category\_Potato chips})
\end{bmatrix} = \begin{bmatrix}
0.386 \\
0.189 \\
0.161 \\
\vdots \\
0.009
\end{bmatrix}$$

Vitamin C is the most important feature (38.6%), followed by micronutrient score (18.9%) and nutrient density (16.1%).

## 5. Loss Function and Optimization

### 5.1 Zero-One Loss

**Definition**:

$$L_{0-1}(y, \hat{y}) = \mathbb{1}(y \neq \hat{y}) = \begin{cases}
0 & \text{if } y = \hat{y} \\
1 & \text{if } y \neq \hat{y}
\end{cases}$$

The 0-1 loss penalizes misclassifications uniformly, regardless of confidence. It directly corresponds to classification error rate.

**Empirical Risk** (training error):

$$R_{\text{emp}}(h) = \frac{1}{n} \sum_{i=1}^{n} L_{0-1}(y_i, h(\mathbf{x}_i))$$

**Expected Risk** (generalization error):

$$R(h) = \mathbb{E}_{(\mathbf{x}, y) \sim P}[L_{0-1}(y, h(\mathbf{x}))]$$

The goal is to minimize expected risk, which is estimated by test error.

**NutriSolve Performance**:

- Training Error: $R_{\text{emp}}(h_{\text{RF}}) = 1 - 0.9390 = 0.0610$ (6.10%)
- Test Error: $\hat{R}(h_{\text{RF}}) = 1 - 0.9512 = 0.0488$ (4.88%)

The model generalizes well (test error < training error), indicating no overfitting despite the small dataset.

### 5.2 Gini Impurity Minimization

Each tree optimizes local Gini impurity at each split:

$$(f^*, \tau^*) = \arg\min_{f, \tau} \left( \frac{|\mathcal{S}_{N_L}|}{|\mathcal{S}_N|} G(N_L) + \frac{|\mathcal{S}_{N_R}|}{|\mathcal{S}_N|} G(N_R) \right)$$

**Equivalently** (maximizing Gini gain):

$$(f^*, \tau^*) = \arg\max_{f, \tau} \Delta G(N, f, \tau)$$

The greedy split selection ensures each partition increases purity, leading to accurate leaf predictions.

### 5.3 Ensemble Risk Decomposition

The expected test error of the ensemble can be decomposed:

$$\mathbb{E}[\text{Error}_{\text{ensemble}}] = \text{Bias}^2 + \text{Variance} + \sigma^2_{\epsilon}$$

**Bias**:

$$\text{Bias}(h_{\text{RF}}) = \mathbb{E}_{\mathcal{D}}[\hat{y}_{\text{RF}}(\mathbf{x})] - y^*(\mathbf{x})$$

**Variance**:

$$\text{Variance}(h_{\text{RF}}) = \mathbb{E}_{\mathcal{D}}[(\hat{y}_{\text{RF}}(\mathbf{x}) - \mathbb{E}_{\mathcal{D}}[\hat{y}_{\text{RF}}(\mathbf{x})])^2]$$

**Bagging Variance Reduction**: For ensemble of $T$ independent trees with variance $\sigma^2$:

$$\text{Var}(\bar{h}_{\text{ensemble}}) = \frac{\sigma^2}{T}$$

With correlation $\rho$ between trees:

$$\text{Var}(\bar{h}_{\text{ensemble}}) = \rho \sigma^2 + \frac{1-\rho}{T} \sigma^2$$

**NutriSolve**: $T = 50$, $\rho \approx 0.6$ (estimated) → Variance reduced by factor of ~8.

Bootstrap sampling and feature bagging decorrelate trees, reducing variance while maintaining low bias.

## 6. Complete Pipeline: Composite Function

### 6.1 End-to-End Transformation

The complete NutriSolve ML pipeline is a composition of preprocessing and prediction functions:

$$\hat{y} = f_{\text{pipeline}}(\mathbf{x}_{\text{raw}}) = h_{\text{RF}} \circ \phi_{\text{select}} \circ \phi_{\text{encode}} \circ \phi_{\text{engineer}} \circ \phi_{\text{impute}}(\mathbf{x}_{\text{raw}})$$

**Step-by-Step**:
1. **Imputation**: $\mathbf{x}_1 = \phi_{\text{impute}}(\mathbf{x}_{\text{raw}}) \in \mathbb{R}^8$
2. **Engineering**: $\mathbf{x}_2 = \phi_{\text{engineer}}(\mathbf{x}_1) \in \mathbb{R}^{13}$
3. **Encoding**: $\mathbf{x}_3 = \phi_{\text{encode}}(\mathbf{x}_2) \in \mathbb{R}^{69}$
4. **Selection**: $\mathbf{x}_4 = \phi_{\text{select}}(\mathbf{x}_3) \in \mathbb{R}^{9}$
5. **Classification**: $\hat{y} = h_{\text{RF}}(\mathbf{x}_4) \in \{0, 1\}$

### 6.2 Dimensionality Transformation

$$\mathbb{R}^8 \xrightarrow{\phi_{\text{impute}}} \mathbb{R}^8 \xrightarrow{\phi_{\text{engineer}}} \mathbb{R}^{13} \xrightarrow{\phi_{\text{encode}}} \mathbb{R}^{69} \xrightarrow{\phi_{\text{select}}} \mathbb{R}^9 \xrightarrow{h_{\text{RF}}} \{0, 1\}$$

The feature space expands from 8 to 69 dimensions through engineering and encoding, then contracts to 9 dimensions via feature selection before final classification. This expansion-contraction strategy captures complex interactions while maintaining model interpretability.

## 7. Computational Complexity

### 7.1 Training Complexity

**Preprocessing**: $O(n \cdot d_{\text{raw}} \log n)$ for median computation (sorting)

**Feature Engineering**: $O(n \cdot d_{\text{raw}})$ for element-wise operations

**Feature Selection**: $O(n \cdot d_{\text{full}} \cdot c)$ for chi-squared test ($c = 2$ classes)

**Single Tree Training**: $O(n \log n \cdot m \cdot \text{depth})$, where $m = \sqrt{d} = 3$ features per split

**Random Forest Training**:

$$O_{\text{RF}} = O(T \cdot n \log n \cdot m \cdot \text{depth}) = O(50 \cdot 164 \log 164 \cdot 3 \cdot 5) \approx O(600,000)$$

**Total Training Complexity**:

$$O_{\text{total}} = O(n d \log n) + O(T n \log n \cdot m \cdot \text{depth}) = O(T n \log n \cdot m \cdot \text{depth})$$

**Empirical**: 1.8 seconds on standard CPU

The complexity is dominated by tree construction. The small dataset and moderate tree depth enable fast training.

### 7.2 Inference Complexity

**Preprocessing**: $O(d_{\text{raw}})$ (constant time for single sample)

**Single Tree Prediction**: $O(\text{depth}) = O(5)$ (traverse root to leaf)

**Random Forest Prediction**:

$$O_{\text{inference}} = O(d_{\text{raw}}) + O(T \cdot \text{depth}) = O(50 \cdot 5) = O(250)$$

**Empirical**: <1 millisecond per sample

Inference is extremely fast, enabling real-time predictions suitable for web applications.

## Summary

The NutriSolve ML pipeline is mathematically formulated as a composite function:

$$f_{\text{pipeline}} = h_{\text{RF}} \circ \phi_{\text{select}} \circ \phi_{\text{encode}} \circ \phi_{\text{engineer}} \circ \phi_{\text{impute}}$$

transforming raw 8-dimensional nutritional data through sequential preprocessing stages:

1. **Median imputation** of missing values
2. **Feature engineering** with 5 derived features (nutrient density, protein ratio, energy density, micronutrient score)
3. **One-hot encoding** of 53 food categories
4. **Chi-squared feature selection** reducing to 9 dimensions (86.6% reduction)
5. **Random Forest classification** with 50 decision trees

**Key Parameters**:
- Trees: $T = 50$
- Max depth: 5
- Min samples split: 5
- Min samples leaf: 3
- Features per split: $m = 3$ (out of 9)

**Performance**:
- Training accuracy: 93.90%
- Test accuracy: 95.12%
- F1-Score (macro): 0.9458
- ROC-AUC: 0.9815

**Complexity**:
- Training: $O(600,000)$ operations (~1.8 seconds)
- Inference: $O(250)$ operations (<1 millisecond)

The model achieves excellent generalization on a small dataset through careful regularization, feature selection, and ensemble methods.